In [36]:
import os
import json
import pandas as pd
import traceback


In [37]:
from dotenv import load_dotenv
load_dotenv()

True

In [38]:
groq=os.getenv('GROQ_API_KEY')


In [39]:
groq

'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'

In [40]:

from langchain_groq import ChatGroq
llm_groq = ChatGroq(temperature=0,api_key=groq, model_name="llama3-8b-8192")

In [41]:
llm_groq

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001B760848990>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001B760823090>, model_name='llama3-8b-8192', temperature=1e-08, groq_api_key=SecretStr('**********'))

In [42]:

from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chains import SequentialChain
from langchain.callbacks import get_openai_callback
from langchain import HuggingFaceHub

import os
import json
import traceback
import pandas as pd
from dotenv import load_dotenv
from langchain.document_loaders import PyPDFLoader
import PyPDF2

In [43]:
TEMPLATE='''
Text:{text}
you are expert MCQ maker. Given the above text, it is your job to\
create a quiz of {number} multiple choice questions for {subject} students\
in {tone} tone. \
Make sure the questions are not repeated and check all the questions to be confirming the text as well.\
Make sure to format your response as per below json format named as RESPONSE_JSON and use it as a guide.\
Ensure to make {number} and the output response should be in json format only as per below format\

### RESPONSE_JSON\
{response_json}
'''



In [44]:
quiz_generation_prompt=PromptTemplate(input_variables=['text','number','subject','tone','response_json'],
                                      template=TEMPLATE
                                      )

In [45]:
quiz_chain=LLMChain(llm=llm_groq,prompt=quiz_generation_prompt,output_key='quiz',verbose=True)

In [46]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. \
Only use at max 50 words for complexity analysis. \
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities\
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [47]:
quiz_evaluation_prompt=PromptTemplate(input_variables=['quiz','subject'],template=TEMPLATE2)

In [48]:
review_chain=LLMChain(llm=llm_groq,prompt=quiz_evaluation_prompt,output_key='review',verbose=True)

In [49]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain,review_chain],
                                        input_variables=['text','number','subject','tone','response_json'],
                                        output_variables=['quiz','review'],verbose=True)

In [50]:
file_path=r"C:\Users\jaipanch\mcqgenerator\data.txt"

In [51]:
with open(file_path,'r') as file:
    Text=file.read()

In [52]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}


type(RESPONSE_JSON)
type(json.dumps(RESPONSE_JSON))

str

In [53]:
NUMBER=3
SUBJECT="machine learning"
TONE="difficult"

with get_openai_callback() as cb:
    response=generate_evaluate_chain.invoke(
        {
            "text": Text,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
        )



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]

ML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field's methods.

The mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [ ]:
print(f"Total Tokens:{cb.total_tokens}")
print(f"Prompt Tokens:{cb.prompt_tokens}")
print(f"Completion Tokens:{cb.completion_tokens}")
print(f"Total Cost:{cb.total_cost}")

Total Tokens:2065
Prompt Tokens:1439
Completion Tokens:626
Total Cost:0.0


In [ ]:
response

{'text': 'Machine learning (ML) is a field of study in artificial intelligence concerned with the development and study of statistical algorithms that can learn from data and generalize to unseen data, and thus perform tasks without explicit instructions.[1] Recently, artificial neural networks have been able to surpass many previous approaches in performance.[2][3]\n\nML finds application in many fields, including natural language processing, computer vision, speech recognition, email filtering, agriculture, and medicine.[4][5] When applied to business problems, it is known under the name predictive analytics. Although not all machine learning is statistically based, computational statistics is an important source of the field\'s methods.\n\nThe mathematical foundations of ML are provided by mathematical optimization (mathematical programming) methods. Data mining is a related (parallel) field of study, focusing on exploratory data analysis (EDA) through unsupervised learning.[7][8]\n

In [ ]:
jsontodict=json.loads(response['quiz'][51:])
jsontodict

{'1': {'mcq': 'What is the primary goal of machine learning?',
  'options': {'a': 'To perform tasks without explicit instructions',
   'b': 'To analyze and visualize data',
   'c': 'To develop and study statistical algorithms',
   'd': 'To create artificial intelligence'},
  'correct': 'c'},
 '2': {'mcq': "Who coined the term 'machine learning' in 1959?",
  'options': {'a': 'Arthur Samuel',
   'b': 'Donald Hebb',
   'c': 'Walter Pitts',
   'd': 'Alan Turing'},
  'correct': 'a'},
 '3': {'mcq': 'What is the primary objective of machine learning algorithms?',
  'options': {'a': 'To classify data based on models',
   'b': 'To make predictions for future outcomes',
   'c': 'To analyze and visualize data',
   'd': 'To create artificial intelligence'},
  'correct': 'b'}}

In [ ]:
quiz_table_data=[]

for key,value in jsontodict.items():
    mcq=value['mcq']
    options=' | '.join([
        f'{option.upper()}: {option_value}' for option,option_value in value['options'].items()
    ])
    correct=value['correct']
    quiz_table_data.append({'MCQ':mcq,'Choices':options,'Correct':correct})

In [ ]:
for i in quiz_table_data:
    print(i)

{'MCQ': 'What is the primary goal of machine learning?', 'Choices': 'A: To perform tasks without explicit instructions | B: To analyze and visualize data | C: To develop and study statistical algorithms | D: To create artificial intelligence', 'Correct': 'c'}
{'MCQ': "Who coined the term 'machine learning' in 1959?", 'Choices': 'A: Arthur Samuel | B: Donald Hebb | C: Walter Pitts | D: Alan Turing', 'Correct': 'a'}
{'MCQ': 'What is the primary objective of machine learning algorithms?', 'Choices': 'A: To classify data based on models | B: To make predictions for future outcomes | C: To analyze and visualize data | D: To create artificial intelligence', 'Correct': 'b'}


In [ ]:
df=pd.DataFrame(quiz_table_data)

In [ ]:
df.to_csv('Machine_Learning.csv',index=False)

In [ ]:
df

,MCQ,Choices,Correct
0,What is the primary goal of machine learning?,A: To perform tasks without explicit instructi...,c
1,Who coined the term 'machine learning' in 1959?,A: Arthur Samuel | B: Donald Hebb | C: Walter ...,a
2,What is the primary objective of machine learn...,A: To classify data based on models | B: To ma...,b
